In [ ]:
import math
import pandas as pd
import numpy as np

CENTER_LON = -121.7493613
CENTER_LAT =  38.5411082
DT_DEFAULT_S = 2.5   # sampling interval used in your generator

def meters_per_degree(lat_deg: float):
    lat = math.radians(lat_deg)
    m_per_deg_lat = 111_132.954 - 559.822 * math.cos(2*lat) + 1.175 * math.cos(4*lat)
    m_per_deg_lon = (math.pi/180) * 6_378_137.0 * math.cos(lat)
    return m_per_deg_lat, m_per_deg_lon

M_PER_DEG_LAT, M_PER_DEG_LON = meters_per_degree(CENTER_LAT)

# Outward-motion speed threshold for Model B outward_flag (tune later if desired)
SPEED_THRESH_MPS = 2.0   # 2.0 ≈ runners only; use 2.7 to be stricter


In [2]:
# Load your short file; change the path if needed
df = pd.read_csv(r"C:\Hackathon\gunshot\expanded_gunshot_sim.csv")

# Convert to local meters relative to the fixed center (inverse of your meters_to_latlon)
df["x_t_m"] = (df["lon"] - CENTER_LON) * M_PER_DEG_LON
df["y_t_m"] = (df["lat"] - CENTER_LAT) * M_PER_DEG_LAT

# Sort so diffs/velocities are correct
df = df.sort_values(["phone_id", "t"]).reset_index(drop=True)


In [3]:
# Time delta per device (protect against missing first-diff)
df["dt_s"] = df.groupby("phone_id")["t"].diff()
df["dt_s"] = df["dt_s"].where(df["dt_s"] > 0, DT_DEFAULT_S)  # replace NaN/<=0 with default

# Position diffs in meters
df["dx_m"] = df.groupby("phone_id")["x_t_m"].diff().fillna(0.0)
df["dy_m"] = df.groupby("phone_id")["y_t_m"].diff().fillna(0.0)

# Velocities (m/s)
df["vx_t_mps"] = df["dx_m"] / df["dt_s"]
df["vy_t_mps"] = df["dy_m"] / df["dt_s"]

# Minimal predictor table for Model A (keep keys for joining labels later)
features_A = df[["phone_id", "t", "x_t_m", "y_t_m", "vx_t_mps", "vy_t_mps"]].copy()

# If you truly want ONLY predictors (no keys), comment the line above and use this instead:
# features_A = df[["x_t_m", "y_t_m", "vx_t_mps", "vy_t_mps"]].copy()

features_A.head()


,phone_id,t,x_t_m,y_t_m,vx_t_mps,vy_t_mps
0,100000000000008,0.0,-20.470981,-19.655965,0.000000,0.000000
1,100000000000008,2.5,-16.038257,-15.458800,1.773089,1.678866
2,100000000000008,5.0,-11.480153,-11.398173,1.823242,1.624251
3,100000000000008,7.5,-6.944687,-7.312014,1.814186,1.634463
4,100000000000008,10.0,-2.360462,-3.280249,1.833690,1.612706


In [4]:
# Speed and radial velocity for each person at each tick
df["speed_mps"] = np.sqrt(df["vx_t_mps"]**2 + df["vy_t_mps"]**2)

# Distance from center; avoid divide-by-zero
r = np.sqrt(df["x_t_m"]**2 + df["y_t_m"]**2)
r_safe = r.replace(0, np.nan)

# Radial velocity = projection of v onto the center->person direction
df["radial_mps"] = (df["vx_t_mps"] * df["x_t_m"] + df["vy_t_mps"] * df["y_t_m"]) / r_safe
df["radial_mps"] = df["radial_mps"].fillna(0.0)  # if exactly at center, treat as 0

# Outward mover = positive radial AND fast enough
df["outward_flag"] = (df["radial_mps"] > 0) & (df["speed_mps"] >= SPEED_THRESH_MPS)

# Aggregate per frame (timestamp t)
grp = df.groupby("t", as_index=False)

# outward_fraction = (# outward) / (# active)
frame_counts = grp["phone_id"].count().rename(columns={"phone_id": "active"})
outward_counts = grp["outward_flag"].sum().rename(columns={"outward_flag": "outward"})
agg = frame_counts.merge(outward_counts, on="t")
agg["outward_fraction"] = agg["outward"] / agg["active"]

# mean_outward_speed among outward movers; if none, use 0.0
mean_out_speed = df[df["outward_flag"]].groupby("t")["speed_mps"].mean().reindex(agg["t"]).fillna(0.0).reset_index(drop=True)
agg["mean_outward_speed_mps"] = mean_out_speed

# Minimal predictor table for Model B (keep t for alignment)
features_B = agg[["t", "outward_fraction", "mean_outward_speed_mps"]].copy()

# If you truly want ONLY predictors (no t), comment the line above and use this instead:
# features_B = agg[["outward_fraction", "mean_outward_speed_mps"]].copy()

features_B.head()


,t,outward_fraction,mean_outward_speed_mps
0,0.0,0.00,0.000000
1,2.5,0.00,0.000000
2,5.0,0.00,0.000000
3,7.5,0.02,2.505238
4,10.0,0.30,2.407130


In [ ]:
features_A.to_csv(r"C:\Hackathon\gunshot\modelA_predictors.csv", index=False)
features_B.to_csv(r"C:\Hackathon\gunshot\modelB_predictors.csv", index=False)